In [2]:
import torch
import torchvision.transforms as transforms
import torchvision
from PIL import Image
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt 
import cv2
import json
import os
import torch.nn as nn
import pickle
import tqdm

In [3]:
# Check if PyTorch is using the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch is using: {device}")

# Print PyTorch version
print(f"PyTorch version: {torch.__version__}")

# Test a simple PyTorch tensor operation on the GPU
x = torch.tensor([2.0, 3.0], device=device)
y = torch.tensor([4.0, 5.0], device=device)
result = x * y
print(f"Result of tensor operation: {result}")

PyTorch is using: cuda
PyTorch version: 2.2.1+cu121
Result of tensor operation: tensor([ 8., 15.], device='cuda:0')


### Perturbations on Normal ResNet50

In [5]:
model = models.resnet50(pretrained=True)
model.eval()

c:\Users\afxwo\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\afxwo\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [30]:
LIMIT_PER_CLASS = 100
EPS = 0.005
STEPS = 20

In [23]:
with open("imagenet_class_index.json", "r") as jf:
    class_index_json = json.load(jf)
print(class_index_json)

{'0': ['n01440764', 'tench'], '1': ['n01443537', 'goldfish'], '2': ['n01484850', 'great_white_shark'], '3': ['n01491361', 'tiger_shark'], '4': ['n01494475', 'hammerhead'], '5': ['n01496331', 'electric_ray'], '6': ['n01498041', 'stingray'], '7': ['n01514668', 'cock'], '8': ['n01514859', 'hen'], '9': ['n01518878', 'ostrich'], '10': ['n01530575', 'brambling'], '11': ['n01531178', 'goldfinch'], '12': ['n01532829', 'house_finch'], '13': ['n01534433', 'junco'], '14': ['n01537544', 'indigo_bunting'], '15': ['n01558993', 'robin'], '16': ['n01560419', 'bulbul'], '17': ['n01580077', 'jay'], '18': ['n01582220', 'magpie'], '19': ['n01592084', 'chickadee'], '20': ['n01601694', 'water_ouzel'], '21': ['n01608432', 'kite'], '22': ['n01614925', 'bald_eagle'], '23': ['n01616318', 'vulture'], '24': ['n01622779', 'great_grey_owl'], '25': ['n01629819', 'European_fire_salamander'], '26': ['n01630670', 'common_newt'], '27': ['n01631663', 'eft'], '28': ['n01632458', 'spotted_salamander'], '29': ['n01632777', 

In [24]:
class_indices = {}
for k, v in class_index_json.items():
    class_indices[v[0]] = int(k)
print(class_indices)

{'n01440764': 0, 'n01443537': 1, 'n01484850': 2, 'n01491361': 3, 'n01494475': 4, 'n01496331': 5, 'n01498041': 6, 'n01514668': 7, 'n01514859': 8, 'n01518878': 9, 'n01530575': 10, 'n01531178': 11, 'n01532829': 12, 'n01534433': 13, 'n01537544': 14, 'n01558993': 15, 'n01560419': 16, 'n01580077': 17, 'n01582220': 18, 'n01592084': 19, 'n01601694': 20, 'n01608432': 21, 'n01614925': 22, 'n01616318': 23, 'n01622779': 24, 'n01629819': 25, 'n01630670': 26, 'n01631663': 27, 'n01632458': 28, 'n01632777': 29, 'n01641577': 30, 'n01644373': 31, 'n01644900': 32, 'n01664065': 33, 'n01665541': 34, 'n01667114': 35, 'n01667778': 36, 'n01669191': 37, 'n01675722': 38, 'n01677366': 39, 'n01682714': 40, 'n01685808': 41, 'n01687978': 42, 'n01688243': 43, 'n01689811': 44, 'n01692333': 45, 'n01693334': 46, 'n01694178': 47, 'n01695060': 48, 'n01697457': 49, 'n01698640': 50, 'n01704323': 51, 'n01728572': 52, 'n01728920': 53, 'n01729322': 54, 'n01729977': 55, 'n01734418': 56, 'n01735189': 57, 'n01737021': 58, 'n0173

In [25]:
class_indices_rev = {}
for k, v in class_indices.items():
    class_indices_rev[v] = k
print(class_indices_rev)

{0: 'n01440764', 1: 'n01443537', 2: 'n01484850', 3: 'n01491361', 4: 'n01494475', 5: 'n01496331', 6: 'n01498041', 7: 'n01514668', 8: 'n01514859', 9: 'n01518878', 10: 'n01530575', 11: 'n01531178', 12: 'n01532829', 13: 'n01534433', 14: 'n01537544', 15: 'n01558993', 16: 'n01560419', 17: 'n01580077', 18: 'n01582220', 19: 'n01592084', 20: 'n01601694', 21: 'n01608432', 22: 'n01614925', 23: 'n01616318', 24: 'n01622779', 25: 'n01629819', 26: 'n01630670', 27: 'n01631663', 28: 'n01632458', 29: 'n01632777', 30: 'n01641577', 31: 'n01644373', 32: 'n01644900', 33: 'n01664065', 34: 'n01665541', 35: 'n01667114', 36: 'n01667778', 37: 'n01669191', 38: 'n01675722', 39: 'n01677366', 40: 'n01682714', 41: 'n01685808', 42: 'n01687978', 43: 'n01688243', 44: 'n01689811', 45: 'n01692333', 46: 'n01693334', 47: 'n01694178', 48: 'n01695060', 49: 'n01697457', 50: 'n01698640', 51: 'n01704323', 52: 'n01728572', 53: 'n01728920', 54: 'n01729322', 55: 'n01729977', 56: 'n01734418', 57: 'n01735189', 58: 'n01737021', 59: 'n

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to fit ResNet-50 input size
    transforms.ToTensor(),  # Convert image to tensor
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize image
])

In [11]:
def predict_image(image_path, model, transform):
    # Load image
    image = Image.open(image_path).convert("RGB") 
    # Apply transformations
    image = transform(image).unsqueeze(0)  # Add batch dimension
    # Make prediction
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        return predicted.item()

In [12]:
def predict_image_no_path(image, model, transform):
    # Apply transformations
    image = transform(image).unsqueeze(0)  # Add batch dimension
    # Make prediction
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        return predicted.item()

In [98]:
def generate_adversaries(model, baseImage, delta, classIdx, steps=50, epsilon=0.005):
    for step in range(steps):
        # watch gradients
        baseImage.requires_grad = True
        delta.requires_grad = True
        
        # add perturbation
        adversary = baseImage + delta
        
        # run perturbed image through the model and calculate the loss
        predictions = model(adversary)
        if step % 5 == 0:
            print("Step: {}, Pred: {}..., Confidence: {}".format(step, torch.argmax(predictions, 1).item(), torch.softmax(predictions, dim=1)[0][torch.argmax(predictions, 1).item()].item() * 100))
        loss = torch.nn.CrossEntropyLoss()(predictions, torch.tensor([classIdx]).to(device))
        
        # log loss
        if step % 5 == 0:
            print("Step: {}, Loss: {}...".format(step, loss.item()))
        
        # calculate gradients of loss with respect to perturbation
        model.zero_grad()
        loss.backward()
        
        # update the perturbation vector
        delta_grad = delta.grad.detach().sign()  # take sign of the gradient
        delta.data = (delta + epsilon * delta_grad).clamp(-epsilon, epsilon)  # update delta and clip
        
        # clamp the perturbation vector
        delta.data = torch.clamp(delta.data, -epsilon, epsilon)
        
    return delta

In [18]:
def gen_sample(image_path, filename, classname):
    image = Image.open(image_path)
    print(image_path)
    image = transform(image).unsqueeze(0)

    # some images are not transformable normally?
    try:
        # save before img
        newimg = Image.open(image_path)
        init_pred = predict_image(image_path, model, transform)
        newimg = transform(newimg)
        transformed_image_np = newimg.clamp(0, 1).permute(1, 2, 0).numpy()
        transformed_image_pil = Image.fromarray((transformed_image_np * 255).astype(np.uint8))
        
    except TypeError:
        return None

    # create perturbation
    image.requires_grad = True
    delta = torch.zeros_like(image, requires_grad=True)

    classIdx = class_indices[classname]
    adversary = generate_adversaries(model, image, delta, classIdx, steps=STEPS, epsilon=EPS)

    # get perturbation
    adversary_pil = adversary.squeeze().permute(1, 2, 0).detach().numpy()
    adversary_pil = Image.fromarray((adversary_pil * 255).astype(np.uint8))
    

    # create adv image
    adversary_image = (image + adversary).clamp(0, 1).squeeze().permute(1, 2, 0).detach().numpy()
    adversary_image = (adversary_image * 255).astype('uint8')
    adversary_image = Image.fromarray(adversary_image)
    new_pred = predict_image_no_path(adversary_image, model, transform)
    

    if init_pred == new_pred:
        return None
    transformed_image_pil.save(f"results\\inputs\\{classname}\\{filename}")
    adversary_pil.save(f"results\\changes\\{classname}\\{filename}")
    adversary_image.save(f"results\\outputs\\{classname}\\{filename}")

    return 1


In [19]:
def gen_sample_set(directory_path):
    classfolders = [fol for fol in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, fol))]
    for classname in classfolders:
        cnt = 0
        print(classname)
        sdpath = os.path.join(directory_path, classname)
        files = [os.path.join(sdpath, fil) for fil in os.listdir(sdpath)]
        for file in files:
            filename = file.split("\\")[-1]
            prediction = predict_image(file, model, transform)
            # normal pred is target class
            if class_indices_rev[prediction] == classname:
                ret = gen_sample(file, filename, classname)
                cnt += 1 if ret else 0
            if cnt >= LIMIT_PER_CLASS:
                break

In [22]:
# create perturbation set
gen_sample_set("..\\imagenette2\\imagenette2\\train")

n03000684
..\imagenette2\imagenette2\train\n03000684\ILSVRC2012_val_00000537.JPEG
Step: 0, Pred: 491..., Confidence: 99.93864297866821
Step: 0, Loss: 0.0006137394811958075...
Step: 5, Pred: 730..., Confidence: 99.2731511592865
Step: 5, Loss: 13.54517650604248...
Step: 10, Pred: 730..., Confidence: 99.87812638282776
Step: 10, Loss: 16.809389114379883...
Step: 15, Pred: 730..., Confidence: 99.92725253105164
Step: 15, Loss: 17.887088775634766...
..\imagenette2\imagenette2\train\n03000684\ILSVRC2012_val_00005506.JPEG
Step: 0, Pred: 491..., Confidence: 99.14592504501343
Step: 0, Loss: 0.008577410131692886...
Step: 5, Pred: 723..., Confidence: 99.92969036102295
Step: 5, Loss: 18.474082946777344...
Step: 10, Pred: 723..., Confidence: 99.99996423721313
Step: 10, Loss: 27.721824645996094...
Step: 15, Pred: 723..., Confidence: 100.0
Step: 15, Loss: 31.030221939086914...
..\imagenette2\imagenette2\train\n03000684\ILSVRC2012_val_00006043.JPEG
Step: 0, Pred: 491..., Confidence: 35.29136776924133
St

## Attempt to load early-exit classification head onto resnet50

In [42]:
newmodel = models.resnet50(pretrained=True)

c:\Users\afxwo\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\afxwo\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [43]:
with open("ll_025.pkl", 'rb') as f:
    ll = pickle.load(f)

In [46]:
newl = nn.Linear(in_features=2048, out_features=10, bias=True)

In [47]:
newl.load_state_dict(ll)

<All keys matched successfully>

In [48]:
newmodel.fc = newl

In [49]:
newmodel

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [88]:
def test(directory_path, model):
    classfolders = [fol for fol in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, fol))]
    for classname in classfolders:
        cnt = 0
        print(classname)
        sdpath = os.path.join(directory_path, classname)
        files = [os.path.join(sdpath, fil) for fil in os.listdir(sdpath)]
        for file in files:
            filename = file.split("\\")[-1]
            image = Image.open(file).convert("RGB") 
            # Apply transformations
            image = transform(image).unsqueeze(0).to(device)  # Add batch dimension
            # Make prediction
            with torch.no_grad():
                outputs = model(image)
            print(torch.argmax(outputs, dim=1))
            cnt += 1
            if cnt > 10:
                break

## Transfer Learning with ResNet50 on Imagenette to match LEENet

In [9]:
train = torchvision.datasets.ImageFolder('..\\imagenette2\\imagenette2\\train', transform=transform)
val = torchvision.datasets.ImageFolder('..\\imagenette2\\imagenette2\\val', transform=transform)

In [10]:
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 3
model.to(device)

train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=32, shuffle=True)


for epoch in tqdm.tqdm(range(num_epochs), desc='Epochs'):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train)}")



c:\Users\afxwo\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\afxwo\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epochs:  33%|███▎      | 1/3 [01:24<02:48, 84.24s/it]

Epoch 1/3, Loss: 0.012095728010798423


Epochs:  67%|██████▋   | 2/3 [02:13<01:03, 63.56s/it]

Epoch 2/3, Loss: 0.0034997648648915315


Epochs: 100%|██████████| 3/3 [03:06<00:00, 62.21s/it]

Epoch 3/3, Loss: 0.00276119855748605


In [11]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the validation set: {100 * correct / total}%')

Accuracy on the validation set: 99.18471337579618%


In [12]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [92]:
with open("transfer_learn.pkl", 'wb') as f:
    pickle.dump(model, f)

In [14]:
imagenette_classes = [0, 217, 482, 491, 497, 566, 569, 571, 574, 701]
update_class_indices = {}
for i in range(len(imagenette_classes)):
    update_class_indices[imagenette_classes[i]] = i
print(update_class_indices)

{0: 0, 217: 1, 482: 2, 491: 3, 497: 4, 566: 5, 569: 6, 571: 7, 574: 8, 701: 9}


In [15]:
update_class_indices_rev = {}
for i in range(len(imagenette_classes)):
    update_class_indices_rev[i] = imagenette_classes[i]
print(update_class_indices_rev)

{0: 0, 1: 217, 2: 482, 3: 491, 4: 497, 5: 566, 6: 569, 7: 571, 8: 574, 9: 701}


In [16]:
def generate_adversaries_transfer(model, baseImage, delta, classIdx, steps=50, epsilon=0.005):
    for step in range(steps):
        # watch gradients
        baseImage.requires_grad = True
        delta.requires_grad = True
        
        # add perturbation
        adversary = baseImage + delta
        
        # run perturbed image through the model and calculate the loss
        predictions = model(adversary)
        if step % 5 == 0:
            print("Step: {}, Pred: {}..., Confidence: {}".format(step, torch.argmax(predictions, 1).item(), torch.softmax(predictions, dim=1)[0][torch.argmax(predictions, 1).item()].item() * 100))
        loss = torch.nn.CrossEntropyLoss()(predictions, torch.tensor([classIdx]).to(device))
        
        # log loss
        if step % 5 == 0:
            print("Step: {}, Loss: {}...".format(step, loss.item()))
        
        # calculate gradients of loss with respect to perturbation
        model.zero_grad()
        loss.backward()
        
        # update the perturbation vector
        delta_grad = delta.grad.detach().sign()  # take sign of the gradient
        delta.data = (delta + epsilon * delta_grad).clamp(-epsilon, epsilon)  # update delta and clip
        
        # clamp the perturbation vector
        delta.data = torch.clamp(delta.data, -epsilon, epsilon)
        
    return delta

In [17]:
def predict_image_transfer(image_path, model, transform):
    # Load image
    image = Image.open(image_path).convert("RGB") 
    # Apply transformations
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    # Make prediction
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        return predicted.item()

In [18]:
def predict_image_no_path_transfer(image, model, transform):
    # Apply transformations
    model.to(device)
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    # Make prediction
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        return predicted.item()

In [40]:
def gen_sample_transfer(image_path, filename, classname):
    image = Image.open(image_path)
    print(image_path)
    image = transform(image).unsqueeze(0).to(device)

    # some images are not transformable normally?
    try:
        # save before img
        newimg = Image.open(image_path)
        init_pred = predict_image_transfer(image_path, model, transform)
        newimg = transform(newimg)
        transformed_image_np = newimg.clamp(0, 1).permute(1, 2, 0).numpy()
        transformed_image_pil = Image.fromarray((transformed_image_np * 255).astype(np.uint8))
        
    except TypeError:
        return None

    # create perturbation
    image.requires_grad = True
    delta = torch.zeros_like(image, requires_grad=True).to(device)

    classIdx = update_class_indices[class_indices[classname]]
    adversary = generate_adversaries_transfer(model, image, delta, classIdx, steps=STEPS, epsilon=EPS)

    # get perturbation
    adversary_pil = adversary.squeeze().permute(1, 2, 0).cpu().detach().numpy()
    adversary_pil = Image.fromarray((adversary_pil * 255).astype(np.uint8))
    

    # create adv image
    adversary_image = (image + adversary).clamp(0, 1).squeeze().permute(1, 2, 0).cpu().detach().numpy()
    adversary_image = (adversary_image * 255).astype('uint8')
    adversary_image = Image.fromarray(adversary_image)
    new_pred = predict_image_no_path_transfer(adversary_image, model, transform)
    

    if init_pred == new_pred:
        return None
    transformed_image_pil.save(f"results_transfer\\inputs\\{classname}\\{filename}", quality=100)
    adversary_pil.save(f"results_transfer\\changes\\{classname}\\{filename}", quality=100)
    adversary_image.save(f"results_transfer\\outputs\\{classname}\\{filename}", quality=100)

    return 1

In [41]:
def gen_sample_set_transfer(directory_path):
    classfolders = [fol for fol in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, fol))]
    for classname in classfolders:
        cnt = 0
        print(classname)
        sdpath = os.path.join(directory_path, classname)
        files = [os.path.join(sdpath, fil) for fil in os.listdir(sdpath)]
        for file in files:
            filename = file.split("\\")[-1]
            prediction = predict_image_transfer(file, model, transform)
            # normal pred is target class
            if class_indices_rev[update_class_indices_rev[prediction]] == classname:
                ret = gen_sample_transfer(file, filename, classname)
                cnt += 1 if ret else 0
            if cnt >= LIMIT_PER_CLASS:
                break

In [42]:
gen_sample_set_transfer("..\\imagenette2\\imagenette2\\train")

n01440764
..\imagenette2\imagenette2\train\n01440764\ILSVRC2012_val_00000293.JPEG
Step: 0, Pred: 0..., Confidence: 97.2394347190857
Step: 0, Loss: 0.02799382619559765...
Step: 5, Pred: 9..., Confidence: 99.33862090110779
Step: 5, Loss: 8.682951927185059...
Step: 10, Pred: 9..., Confidence: 99.99309778213501
Step: 10, Loss: 13.663358688354492...
Step: 15, Pred: 9..., Confidence: 99.99890327453613
Step: 15, Loss: 15.55126953125...
..\imagenette2\imagenette2\train\n01440764\ILSVRC2012_val_00002138.JPEG
Step: 0, Pred: 0..., Confidence: 94.5752501487732
Step: 0, Loss: 0.05577438324689865...
Step: 5, Pred: 3..., Confidence: 98.5300600528717
Step: 5, Loss: 12.98268985748291...
Step: 10, Pred: 3..., Confidence: 99.99197721481323
Step: 10, Loss: 18.0021915435791...
Step: 15, Pred: 3..., Confidence: 99.99841451644897
Step: 15, Loss: 19.499713897705078...
..\imagenette2\imagenette2\train\n01440764\ILSVRC2012_val_00003014.JPEG
Step: 0, Pred: 0..., Confidence: 86.64010167121887
Step: 0, Loss: 0.143

In [32]:
image_path = "C:\\CS8803 SPD\\Project\\gitrepo\\imagenette2\\imagenette2\\train\\n01440764\\ILSVRC2012_val_00000293.JPEG"
image = Image.open(image_path)
print(image_path)
image = transform(image).unsqueeze(0).to(device)
image.requires_grad = True
delta = torch.zeros_like(image, requires_grad=True).to(device)
model.to(device)
classIdx = 0
adversary = generate_adversaries_transfer(model, image, delta, classIdx, steps=STEPS, epsilon=EPS)
adversary_image = (image + adversary).clamp(0, 1).squeeze().permute(1, 2, 0).cpu().detach().numpy()
adversary_image = (adversary_image * 255).astype('uint8')
adversary_image = Image.fromarray(adversary_image)
new_pred = predict_image_no_path_transfer(adversary_image, model, transform)

C:\CS8803 SPD\Project\gitrepo\imagenette2\imagenette2\train\n01440764\ILSVRC2012_val_00000293.JPEG
Step: 0, Pred: 0..., Confidence: 97.2394347190857
Step: 0, Loss: 0.02799382619559765...
Step: 5, Pred: 9..., Confidence: 99.48517680168152
Step: 5, Loss: 8.696564674377441...
Step: 10, Pred: 9..., Confidence: 99.9942421913147
Step: 10, Loss: 14.230751037597656...
Step: 15, Pred: 9..., Confidence: 99.99916553497314
Step: 15, Loss: 16.357378005981445...


In [38]:
adversary_image.save(f"test.jpg", quality=100)

In [43]:
with open("transfer_learn.pkl", 'rb') as f:
    tempmodel = pickle.load(f)

tempmodel.eval()
tempmodel.to(device)

temp = Image.open("results_transfer\\outputs\\n01440764\\ILSVRC2012_val_00007197.JPEG")
temp = transform(temp)
temp = temp.unsqueeze(0).to(device)
with torch.no_grad():
    tempout = tempmodel(temp)
    print(tempout)
    temppredclass = torch.argmax(tempout, dim=1).item()
print(temppredclass)

tensor([[ 1.9065,  1.1906, -0.1927, -0.8905, -2.8073, -0.2170, -3.0746, -2.5463,
          6.6498,  0.8638]], device='cuda:0')
8
